In [9]:
import os
from lib.AirtableLib import AirtableData

In [10]:
personal_base_id = 'appYknCML4IMHe9DP'
personal_table_id = 'tblQcjtQ7kB3dJx1m'
airtable_api_key = os.environ.get('AIRTABLE_API_KEY')

In [ ]:
personal_airtable = AirtableData(personal_base_id, personal_table_id, airtable_api_key)
personal_data = personal_airtable.get_data()
print(personal_data)

## Oura Columns Work

In [12]:
import requests
import random
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Any, Union

#### Testing Oura API endpoints

In [13]:
base_url = 'https://api.ouraring.com/v2/usercollection/'
access_token = os.environ.get('OURA_TOKEN')
endpoints = ['daily_activity','daily_readiness', 'daily_sleep', 'daily_spo2', 'daily_stress']
start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

In [14]:
def test_oura_endpoint(endpoint, start_date, end_date):
    url = base_url + endpoint + '?start_date=' + start_date + '&end_date=' + end_date
    r = requests.get(url, headers={'Authorization': 'Bearer ' + access_token})
    return r.json()

In [15]:
# Test Each Endpoint
for endpoint in endpoints:
    data = test_oura_endpoint(endpoint, start_date, end_date)["data"]
    print(endpoint)
    print(f'Records: {len(data)}')
    print(f'Days: {[x["day"] for x in data]}')
    print(data)
    print()

daily_activity
Records: 1
Days: ['2024-01-25']
[{'id': 'a038fd6b-2800-43c1-875c-9f6033f50da4', 'class_5_min': '111111111111111111111111111112334333344433333333443223222222222222222222333232322332232222233322223222222222222222233222323333222222223232222233322332223222222233333333343333333333322322333333333322222322222222222000000002343222222222222222211111111111111111111111111111111111111111111111', 'score': 80, 'active_calories': 468, 'average_met_minutes': 1.53125, 'contributors': {'meet_daily_targets': 43, 'move_every_hour': 100, 'recovery_time': 100, 'stay_active': 66, 'training_frequency': 100, 'training_volume': 98}, 'equivalent_walking_distance': 8354, 'high_activity_met_minutes': 20, 'high_activity_time': 120, 'inactivity_alerts': 0, 'low_activity_met_minutes': 241, 'low_activity_time': 20280, 'medium_activity_met_minutes': 114, 'medium_activity_time': 2220, 'met': {'interval': 60.0, 'items': [1.1, 1.0, 0.9, 0.9, 0.9, 0.9, 1.1, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0

In [16]:
activity_columns = ['score', 'active_calories', 'contributors__meet_daily_targets', 'contributors__move_every_hour',
                    'contributors__recovery_time', 'contributors__stay_active', 'contributors__training_frequency', 
                    'contributors__training_volume', 'steps', 'target_calories', 'target_meters', 'total_calories', 
                    'day', 'high_activity_met_minutes', 'high_activity_time', 'medium_activity_met_minutes',
                    'medium_activity_time', 'low_activity_met_minutes', 'low_activity_time', 'rest_activity_met_minutes',
                    'equivalent_walking_distance', 'inactivity_alerts']

readiness_columns = ['score', 'contributors__activity_balance', 'contributors__body_temperature', 'contributors__hrv_balance',
                    'contributors__previous_day_activity', 'contributors__previous_night', 'contributors__recovery_index', 
                    'contributors__resting_heart_rate', 'contributors__sleep_balance', 'temperature_deviation', 'temperature_trend_deviation']

sleep_columns = ['score', 'contributors__deep_sleep', 'contributors__efficiency', 'contributors__latency', 'contributors__rem_sleep',
                'contributors__restfulness', 'contributors__timing', 'contributors__total_sleep']

spo2_columns = ['spo2_percentage__average']

stress_columns = ['stress_high', 'recovery_high', 'day_summary']

oura_columns = [activity_columns, readiness_columns, sleep_columns, spo2_columns, stress_columns]

In [17]:
oura_columns_with_enpoints = dict(zip(endpoints, oura_columns))
oura_columns_with_enpoints

{'daily_activity': ['score',
  'active_calories',
  'contributors__meet_daily_targets',
  'contributors__move_every_hour',
  'contributors__recovery_time',
  'contributors__stay_active',
  'contributors__training_frequency',
  'contributors__training_volume',
  'steps',
  'target_calories',
  'target_meters',
  'total_calories',
  'day',
  'high_activity_met_minutes',
  'high_activity_time',
  'medium_activity_met_minutes',
  'medium_activity_time',
  'low_activity_met_minutes',
  'low_activity_time',
  'rest_activity_met_minutes',
  'equivalent_walking_distance',
  'inactivity_alerts'],
 'daily_readiness': ['score',
  'contributors__activity_balance',
  'contributors__body_temperature',
  'contributors__hrv_balance',
  'contributors__previous_day_activity',
  'contributors__previous_night',
  'contributors__recovery_index',
  'contributors__resting_heart_rate',
  'contributors__sleep_balance',
  'temperature_deviation',
  'temperature_trend_deviation'],
 'daily_sleep': ['score',
  'co

In [20]:
def read_oura_endpoint(endpoint: str, start_date: str, end_date: str, columns: List[str]) -> pd.DataFrame:
    # Retrieve data from the endpoint
    json_data = test_oura_endpoint(endpoint, start_date, end_date)["data"]

    # Filter out data where 'day' does not equal start_date
    json_data = json_data[0]

    # Create a dictionary of the data
    df_dict = {}
    for col in columns:
        if '__' in col:
            parent = col.split('__')[0]
            child = col.split('__')[1]
            df_dict[col] = json_data[parent][child]
        else:
            df_dict[col] = json_data[col]
            
    return pd.DataFrame(df_dict, index=[0])

In [21]:
# Create DataFrames for each endpoint for each day from 2024-01-10 to today
activity_data = pd.DataFrame()
readiness_data = pd.DataFrame()
sleep_data = pd.DataFrame()
spo2_data = pd.DataFrame()
stress_data = pd.DataFrame()

dates = pd.date_range(start='2021-01-10', end=datetime.now().strftime('%Y-%m-%d'))

for date in dates:
    start_date = date.strftime('%Y-%m-%d')
    end_date = (date + timedelta(days=1)).strftime('%Y-%m-%d')
    activity_data = pd.concat([activity_data, read_oura_endpoint('daily_activity', start_date, end_date, activity_columns)])
    readiness_data = pd.concat([readiness_data, read_oura_endpoint('daily_readiness', start_date, end_date, readiness_columns)])
    sleep_data = pd.concat([sleep_data, read_oura_endpoint('daily_sleep', start_date, end_date, sleep_columns)])
    spo2_data = pd.concat([spo2_data, read_oura_endpoint('daily_spo2', start_date, end_date, spo2_columns)])
    stress_data = pd.concat([stress_data, read_oura_endpoint('daily_stress', start_date, end_date, stress_columns)])

print(activity_data)
print(readiness_data)
print(sleep_data)
print(spo2_data)
print(stress_data)



IndexError: list index out of range